# poetryのインストール・仮想環境のアクティベート

In [ ]:
# # モデルのインポート
import os
import transformers

In [ ]:
# todo : CasualLM と CasualLLM の違いを調べる
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    DataCollatorForSeq2Seq,
)
import torch
from datasets import load_dataset
import peft

# 設定

In [ ]:
trust_remote_code = True
# hugging face cli tokenの設定
os.environ["HUGGINGFACE_TOKEN"] = ""
output_dir = "./output"

# 量子化読み込みの設定

In [ ]:
# 8bit量子化を行う
load_in_8bit = True
load_in_4bit = False

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_8bit=load_in_8bit, load_in_4bit=load_in_4bit
)

In [ ]:
# GPUを使う
device_map = {"": 0}
# bfloat16を使う
torch_dtype = torch.bfloat16

# モデルの読み込み

In [ ]:
# hugging faceのモデルを指定
# use llama2
base_model = ""

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quantization_config,
    device_map=device_map,
    torch_dtype=torch_dtype,
    use_auth_token=trust_remote_code,
    trust_remote_code=trust_remote_code,
)

In [ ]:
# tokenizerの設定
tokenizer = AutoTokenizer.from_pretrained(base_model)
# paddingのトークンはeosとは別にしないといけない
# llama2はeosがpaddingのトークンになっている...どうすれば？？
tokenizer.pad_token = "[PAD]"
tokenizer.padding_side = "left"

# 基本の学習パラメータの設定

In [ ]:
batch_size = 1
gradient_accumulation_steps = 1
learining_rate = 5e-5
num_train_epochs = 1
logging_steps = 100
max_steps = -1
save_steps = 100
save_total_limit = 10
val_size = 120

In [ ]:
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learining_rate,
    num_train_epochs=num_train_epochs,
    logging_steps=logging_steps,
    max_steps=max_steps,
    save_steps=save_steps,
    save_total_limit=save_total_limit,
)

# PEFTの設定

## Lora
 
loraの設定 with peft library

In [ ]:
lora_r: int = 8
lora_alpha: int = 16
lora_dropout: float = 0.05
lora_target_modules: list[str] = None

In [ ]:
lora_config = peft.LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=lora_target_modules,
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CASUAL_LM",
)

## AdaLoraの設定


In [ ]:
# loraの設定
lora_r: int = 8
lora_alpha: int = 16
lora_dropout: float = 0.05
lora_target_modules: list[str] = None

In [ ]:
# adapter特有の設定
target_r: int = 8
init_r: int = 12
tinit_r: int = 0
tfinal_r: int = 0
deltaT: float = 1.0
beta1: float = 0.85
beta2: float = 0.85
orth_reg_weight: float = 0.0
total_step: list[int] = None
rank_patterns: list[str] = None

In [ ]:
adalora_config = peft.AdaLoraConfig(
        # lora setting
        lora_r=lora_r,
        lora_alpha=lora_alpha,
        target_modules=lora_target_modules,
        lora_dropout=lora_dropout,
        bias="none",
        task_type="CASUAL_LM"
        # adalora setting
        target_r=target_r,
        init_r=init_r,
        tinit_r=tinit_r,
        tfinal_r=tfinal_r,
        deltaT=deltaT,
        beta1=beta1,
        beta2=beta2,
        orth_reg_weight=orth_reg_weight,
        total_step=total_step,
        rank_patterns=rank_patterns,
    )

## Prefix-tuningの設定
https://huggingface.co/docs/peft/task_guides/ptuning-seq-classification

In [ ]:
num_virtual_tokens: int = 8

In [ ]:
prefix_config = peft.PrefixTuningConfig(
    num_virtual_tokens=num_virtual_tokens, task_type="CASUAL_LM"
)

# Prompt-tuningの設定


In [ ]:
# TODO : 設定調べる
# これはhugging faceにあったデフォルトの設定
prompt_tuning_init: str = peft.PromptTuningInit.Text  # or "random"
prompt_tuning_init_text: str = ""  # prompttuningのテキストを指定
num_virtual_tokens: int = 8

In [ ]:
prompt_config = peft.PromptTuningConfig(
    prompt_tuning_init=prompt_tuning_init,
    prompt_tuning_init_text=prompt_tuning_init_text,
    num_virtual_tokens=num_virtual_tokens,
    task_type="CASUAL_LM",
)

## P-tuningの設定

In [ ]:
# P-tuningではprompt encoderで設定を行う
num_virtual_tokens: int = 8  # 仮想トークンの数
encoder_hidden_dim: int = 128  # prompt encoderの隠れ層の次元数

In [ ]:
p_config = peft.PromptEncoderConfig(
    num_virtual_tokens=num_virtual_tokens,
    encoder_hidden_dim=encoder_hidden_dim,
    task_type="CASUAL_LM",
)

## IA3の設定

In [ ]:
# 全然わからない
# TODO : 設定調べる
target_modules: list[str] = (None,)
feedforward_modules: list[str] = (None,)
fan_in_fan_out = true
modules_to_save: list[str] = None

In [ ]:
ia3_config = peft.IA3Config(
        target_modules=None,
        feedforward_modules=
        fan_in_fan_out = fan_in_fan_out,
        modules_to_save=None
        task_type="CASUAL_LM",
        )

## Adapter H

## Adapter P 

## Adapter Pararell

# peftモデルの読み込み

In [ ]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# データセットの設定

In [ ]:
def formatting_func(examples: list[dict]):
    """データセットのフォーマットを設定する関数

    Args :
        examples (list[dict]): データセットのdict
    Returns
        (list[str]): フォーマットされたデータセット
    """
    texts = []

    for example in examples:
        if example["input"]:
            text = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. 

                ### Instruction:
                {example["instruction"]}
                
                ### Input:
                {example["input"]}
                
                ### Response:
                {example["output"]}"""
            texts.append(text)
        else:
            text = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request. 

                ### Instruction:
                {example["instruction"]}
                
                ### Response:
                {example["output"]}"""
            texts.append(text)

    return texts

## データセットの読み込み
データセットはjson形式で保存されている必要がある

In [ ]:
data_path = ""
dataset = load_dataset("json", data_files=data_path)

In [ ]:
# trainとtestに分ける
train_val = dataset.train_test_split(test_size=val_size, shuffle=True, seed=42)

In [ ]:
train_data = train_val["train"].shuffle().map(formatting_func)
val_data = train_val["test"].shuffle().map(formatting_func)

In [ ]:
trainer = transformers.Trainer(
        model=model,
        train_dataset=train_data,
        eval_dataset=val_data,
        args = training_args,
        data_collator = DataCollatorForSeq2Seq(tokenizer, pad_to_multiple_of=8, return_tensors="pt" padding=True)
)